# 1a. Calculate <i>P<sup>v</sup></i><sub>sat</sub> for a single melt composition from a dataframe using default options

## Python set-up
You need to install VolFe once on your machine, if you haven't yet. Then we need to import a few Python packages (including VolFe). 

In [1]:
# Install VolFe on your machine.
# pip install VolFe # Remove the first # in this line if you have not installed VolFe on your machine before.

# import python packages
import pandas as pd
import VolFe as vf

## Define the inputs

At a minimum to run a <i>P<sup>v</sup></i><sub>sat</sub> calculations, we need a dataframe of the melt composition, an estimate of oxygen fugacity and Fe in the melt [note 1], and temperature:

- Sample is just the name for this analysis.

- Temperature is in °C.

- Volatile-free melt composition is in wt% oxides [note 2]. All these oxides must be present in the dataframe, so set them to 0. if you have no data for them (although see [note 1] around FeOT).

- H2O is all hydrogen in the melt reported as H<sub>2</sub>O in wt% [note 3]. 

- CO2ppm is all carbon in the melt reported as CO<sub>2</sub> in ppm [note 3].

- STppm is all sulfur in the melt reported as S in ppm [note 3].

- Xppm is all "X" in the melt reported as "X" in ppm [note 3]. "X" is an unreactive melt species whose identity can be changed - this is explored in Example 1d.

- Fe3+FeT is the ratio of Fe<sup>3+</sup> to Fe<sub>T</sub> in the melt.

[note 1] In this example we specify oxygen fugacity using Fe<sup>3+</sup>/Fe<sub>T</sub> and Fe in the melt as FeO<sub>T</sub> as is quite common for melt inclusion and matrix glass analyses - other options are possible, shown later in the notebook.

[note 2] It does not matter what the non-volatile oxides sum too - they are renormalised to 100 wt% minus the total of the volatiles (i.e., H<sub>2</sub>O + CO<sub>2</sub> + S<sub>T</sub> + X).

[note 3] The volatile concentrations are absolute for the melt = i.e., the non-volatile melt composition is normalised to 100 wt% minus the volatiles. The volatiles are not added to the oxides and then renormalised.

The following composition is analysis TN273-01D-01-01 from Brounce et al. (2014) with the updated Fe<sup>3+</sup>/Fe<sub>T</sub> from Cottrell et al. (2021), with a temperature chosen as 1200 °C.

In [2]:
# Define the melt composition, fO2 estimate, and T as a dictionary.
my_analysis = {'Sample':'TN273-01D-01-01',
           'T_C': 1200., # Temperature in 'C
           'SiO2': 56.98, # wt%
           'TiO2': 1.66, # wt%
           'Al2O3': 15.52, # wt%
           'FeOT': 9.47, # wt%
           'MnO': 0.24, # wt%
           'MgO': 2.96, # wt%
           'CaO': 6.49, # wt%
           'Na2O': 4.06, # wt%
           'K2O': 0.38, # wt%
           'P2O5': 0.22, # wt%
           'H2O': 1.88, # wt%
           'CO2ppm': 13., # ppm
           'STppm': 362.83, # ppm
           'Xppm': 0., # ppm
           'Fe3FeT': 0.155} # mole or weight fraction (they're the same)

# Turn the dictionary into a pandas dataframe, setting the index to 0.
my_analysis = pd.DataFrame(my_analysis, index=[0])

# Show the DataFrame.
print(my_analysis)

            Sample     T_C   SiO2  TiO2  Al2O3  FeOT   MnO   MgO   CaO  Na2O  \
0  TN273-01D-01-01  1200.0  56.98  1.66  15.52  9.47  0.24  2.96  6.49  4.06   

    K2O  P2O5   H2O  CO2ppm   STppm  Xppm  Fe3FeT  
0  0.38  0.22  1.88    13.0  362.83   0.0   0.155  


For this example we will use the default options in VolFe, which can be found below:

In [3]:
# print default options in VolFe
print(vf.default_models)

                            option
type                              
COH_species     yes_H2_CO_CH4_melt
H2S_m                         True
species X                       Ar
Hspeciation                   none
fO2                       Kress91A
...                            ...
error                          0.1
print status                 False
output csv                    True
setup                        False
high precision               False

[78 rows x 1 columns]


## Run the calculation

### Define <i>f</i><sub>O<sub>2</sub></sub> using Fe<sup>3+</sup>/Fe<sub>T</sub>

And below runs the calculation - it outputs a dataframe of the full results.

In [4]:
# runs the calculation
vf.calc_Pvsat(my_analysis)

<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>


TypeError: type() takes 1 or 3 arguments

### Define <i>f</i><sub>O<sub>2</sub></sub> using ΔFMQ

Alternatively, we can define the oxygen fugacity using ΔFMQ and total iron from Fe<sub>2</sub>O<sub>3,T</sub> ...

In [ ]:
# Define the melt composition, fO2 estimate, and T as a dictionary.
my_analysis = {'Sample':'TN273-01D-01-01',
           'T_C': 1200.,
           'SiO2': 56.98,
           'TiO2': 1.66,
           'Al2O3': 15.52,
           'Fe2O3T': 9.47, # Fe2O3T instead of FeOT
           'MnO': 0.24,
           'MgO': 2.96,
           'CaO': 6.49,
           'Na2O': 4.06,
           'K2O': 0.38,
           'P2O5': 0.22,
           'H2O': 1.88,
           'CO2ppm': 13.,
           'STppm': 362.83,
           'Xppm': 0.,
           'DFMQ': 0.} # DFMQ instead of Fe3+/FeT

# Turn the dictionary into a pandas dataframe, setting the index to 0.
my_analysis = pd.DataFrame(my_analysis, index=[0])

# runs the calculation
vf.calc_Pvsat(my_analysis)

### Define <i>f</i><sub>O<sub>2</sub></sub> using FeO and Fe<sub>2</sub>O<sub>3</sub>

... or FeO and Fe<sub>2</sub>O<sub>3</sub> ...

In [ ]:
# Define the melt composition, fO2 estimate, and T as a dictionary.
my_analysis = {'Sample':'TN273-01D-01-01',
           'T_C': 1200.,
           'SiO2': 56.98,
           'TiO2': 1.66,
           'Al2O3': 15.52,
           'FeO': 8.03, # FeO and Fe2O3 to contraint total Fe and fO2
           'Fe2O3': 2.57, # ^
           'MnO': 0.24,
           'MgO': 2.96,
           'CaO': 6.49,
           'Na2O': 4.06,
           'K2O': 0.38,
           'P2O5': 0.22,
           'H2O': 1.88,
           'CO2ppm': 13.,
           'STppm': 362.83,
           'Xppm': 0.} 

# Turn the dictionary into a pandas dataframe, setting the index to 0.
my_analysis = pd.DataFrame(my_analysis, index=[0])

# runs the calculation
vf.calc_Pvsat(my_analysis)

### Define <i>f</i><sub>O<sub>2</sub></sub> using ΔNNO

... ΔNNO and FeO<sub>T</sub> ...

In [ ]:
# Define the melt composition, fO2 estimate, and T as a dictionary.
my_analysis = {'Sample':'TN273-01D-01-01',
           'T_C': 1200.,
           'SiO2': 56.98,
           'TiO2': 1.66,
           'Al2O3': 15.52,
           'FeOT': 9.47, ###
           'MnO': 0.24,
           'MgO': 2.96,
           'CaO': 6.49,
           'Na2O': 4.06,
           'K2O': 0.38,
           'P2O5': 0.22,
           'H2O': 1.88,
           'CO2ppm': 13.,
           'STppm': 362.83,
           'Xppm': 0.,
           'DNNO': 1.} ###

# Turn the dictionary into a pandas dataframe, setting the index to 0.
my_analysis = pd.DataFrame(my_analysis, index=[0])

# runs the calculation
vf.calc_Pvsat(my_analysis)

### Define <i>f</i><sub>O<sub>2</sub></sub> using S<sup>6+</sup>/S<sub>T</sub>

... or S<sup>6+</sup>/S<sub>T</sub> and FeO<sub>T</sub> ...

In [ ]:
# Define the melt composition, fO2 estimate, and T as a dictionary.
my_analysis = {'Sample':'TN273-01D-01-01',
           'T_C': 1200.,
           'SiO2': 56.98,
           'TiO2': 1.66,
           'Al2O3': 15.52,
           'FeOT': 9.47, 
           'MnO': 0.24,
           'MgO': 2.96,
           'CaO': 6.49,
           'Na2O': 4.06,
           'K2O': 0.38,
           'P2O5': 0.22,
           'H2O': 1.88,
           'CO2ppm': 13.,
           'STppm': 362.83,
           'Xppm': 0.,
           'S6ST': 0.23} ###

# Turn the dictionary into a pandas dataframe, setting the index to 0.
my_analysis = pd.DataFrame(my_analysis, index=[0])

# runs the calculation
vf.calc_Pvsat(my_analysis)

## Potential error: defining <i>f</i><sub>O<sub>2</sub></sub> in too many ways

If we try to specify too many variables to constrain total Fe and <i>f</i><sub>O<sub>2</sub></sub>, we'll get a warning from VolFe telling you it is only using one of the options to calculate things.

In [ ]:
# Define the melt composition, fO2 estimate, and T as a dictionary.
my_analysis = {'Sample':'TN273-01D-01-01',
           'T_C': 1200.,
           'SiO2': 56.98,
           'TiO2': 1.66,
           'Al2O3': 15.52,
           'FeOT': 9.47, ###
           'MnO': 0.24,
           'MgO': 2.96,
           'CaO': 6.49,
           'Na2O': 4.06,
           'K2O': 0.38,
           'P2O5': 0.22,
           'H2O': 1.88,
           'CO2ppm': 13.,
           'STppm': 362.83,
           'Xppm': 0.,
           'DNNO': 1.,
           'Fe3FeT':0.155} ###

# Turn the dictionary into a pandas dataframe, setting the index to 0.
my_analysis = pd.DataFrame(my_analysis, index=[0])

# runs the calculation
vf.calc_Pvsat(my_analysis)